In [1]:
from environment import Environment, Phase
from magiccube import Cube
from agents import A2C, DQN, PPO
import yaml
import numpy as np
import itertools
import torch
import torch.nn as nn

In [2]:
args = yaml.safe_load(open("config.yaml", "r"))
phase = Phase.CROSS

env = Environment(
    method="LBL",
    phase=phase,
    size=3,
    args=args["environment"]
)

agent = DQN(env, args[phase.value])

In [3]:
cube = Cube()
print(cube)

          W  W  W                   
          W  W  W                   
          W  W  W                   
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
 O  O  O  G  G  G  R  R  R  B  B  B 
          Y  Y  Y                   
          Y  Y  Y                   
          Y  Y  Y                   



In [4]:
#env.scramble()
env.step("B'")
print(env.state2)
#print(env.state2.shape)
#print(env.algorithm.status())
print(env.cube)
print(agent.policy_net(agent.state_to_tensor(env.state2)))

[[[ 5  2  0  0 25]
  [ 4  3 -1  1  9]
  [ 1  2  4  2  6]
  [ 0 -1  4  3 20]
  [ 1 -1 -1  4  4]
  [ 2 -1  0  5 24]
  [ 5  3  0  6 19]
  [ 5  2 -1  7 16]
  [ 3  1  5  8  2]
  [-1  2  1  9  7]
  [-1  3 -1 10 10]
  [-1  2  4 11 14]
  [-1 -1  4 12 12]
  [-1 -1  5 13 13]
  [-1  3  0 14 18]
  [-1  2 -1 15 15]
  [-1  1  5 16  5]
  [ 5  2  1 17  8]
  [ 5  3 -1 18 11]
  [ 3  4  1 19  0]
  [ 1 -1  4 20  3]
  [ 0 -1 -1 21 21]
  [ 3 -1  1 22  1]
  [ 4  3  0 23 17]
  [ 5  0 -1 24 22]
  [ 0  4  2 25 23]]]
          Y  Y  Y                   
          W  W  R                   
          O  O  B                   
 G  G  Y  G  G  W  R  G  B  R  R  R 
 R  O  W  R  G  O  Y  R  O  B  B  B 
 G  B  O  B  B  O  Y  G  G  O  O  R 
          W  W  B                   
          Y  Y  Y                   
          W  W  W                   

tensor([[-0.0907, -0.0390, -0.1183, -0.0527, -0.1265, -0.0816,  0.0156,  0.0962,
          0.1045, -0.0003,  0.0097,  0.1079]], grad_fn=<AddmmBackward0>)


In [5]:
hidden_size = 128

embedding = nn.Linear(5, hidden_size)

attention = nn.MultiheadAttention(
    embed_dim=hidden_size,
    num_heads=4,
    batch_first=True
)

layer_norm = nn.LayerNorm(hidden_size)

ffn = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(inplace=True)
)

output_layer = nn.Sequential(
    nn.Linear(hidden_size * 26, hidden_size * 12),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 12, hidden_size * 6),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 6, hidden_size * 3),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size * 3, hidden_size),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size, hidden_size // 2),
    nn.ReLU(inplace=True),
    nn.Linear(hidden_size // 2, 12)
)


In [6]:
state_tensor = torch.from_numpy(env.state2).float()
embedded = embedding(state_tensor)
attn_output, _ = attention(embedded, embedded, embedded)
attn_output = layer_norm(embedded + attn_output)
ffn_output = ffn(attn_output)
output = layer_norm(attn_output + ffn_output)

flattened = output.view(1, -1)
output = output_layer(flattened)

output

tensor([[ 0.0610, -0.0051, -0.0321,  0.0963,  0.0108, -0.0778,  0.0477,  0.0442,
          0.0501, -0.1090,  0.0114, -0.0915]], grad_fn=<AddmmBackward0>)

In [ ]:
# Definiamo un layer lineare: input con 3 feature, output con 2 feature
fc1 = nn.Linear(in_features=3, out_features=256)
fc2 = nn.Linear(in_features=256, out_features=2)

# Esempio di input: batch di 4 esempi, ciascuno con 3 feature
input_matrix = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0]
])  # shape: (4, 3)

# Passaggio della matrice al layer
output = fc1(input_matrix)
print(output)
output = fc2(output)

print("Output:", output)

tensor([[ -0.6882,   1.2118,  -2.9131,  ...,   0.3695,   1.3784,  -0.3020],
        [ -1.2077,   3.4992,  -6.9242,  ...,   1.0029,   3.5215,   0.8250],
        [ -1.7271,   5.7865, -10.9354,  ...,   1.6362,   5.6646,   1.9521],
        [ -2.2465,   8.0739, -14.9465,  ...,   2.2696,   7.8077,   3.0792]],
       grad_fn=<AddmmBackward0>)
Output: tensor([[-0.6831,  0.1601],
        [-1.3377,  0.5216],
        [-1.9924,  0.8831],
        [-2.6470,  1.2446]], grad_fn=<AddmmBackward0>)
